Source: Chapter 14 of Hands-On Machine Learning with Scikit-Learn and TensorFlow 

## Predicting MNIST with RNNs

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.contrib.layers import fully_connected

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# Note that the fully connected layer is connected to the states tensor, which contains only the final state of the RNN (i.e., the 28th output)
logits = fully_connected(states, n_outputs, activation_fn=None)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
               labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [4]:
# load mnist data
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

# Reshape the test data into [batch_size, n_steps, n_inputs] as is expected by the network. We will take care of reshaping the training data in a moment.

X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
# train model
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

(0, 'Train accuracy:', 0.89333332, 'Test accuracy:', 0.91890001)
(1, 'Train accuracy:', 0.95999998, 'Test accuracy:', 0.94190001)
(2, 'Train accuracy:', 0.96666664, 'Test accuracy:', 0.94770002)
(3, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.95819998)
(4, 'Train accuracy:', 0.95999998, 'Test accuracy:', 0.96210003)
(5, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.96240002)
(6, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.96679997)
(7, 'Train accuracy:', 0.98666668, 'Test accuracy:', 0.97180003)
(8, 'Train accuracy:', 0.98666668, 'Test accuracy:', 0.96619999)
(9, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.96520001)
